<a href="https://colab.research.google.com/github/DinurakshanRavichandran/Visio-Glance/blob/OCT-eye-disease-detection/CourseworkModel2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Set Up Your Environment

In [1]:
# Core Libraries
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Average
from tensorflow.keras.models import Model
from sklearn.utils.class_weight import compute_class_weight

# For reproducibility
import random
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

print("Environment ready!")


Environment ready!


In [2]:
import zipfile

# Path to the zipped dataset in your Drive
zip_path = '/content/drive/MyDrive/Machine learning/Dataset/OCT Dataset/Train/archive.zip'

# Set extraction path in Colab
extract_path = '/content/OCT_Dataset'

# Unzipping the dataset
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Dataset unzipped successfully!")



Dataset unzipped successfully!


2. Data Preparation

In [3]:
import os
import random
import shutil

def limit_images_per_class(src_dir, dest_dir, max_images=8000):
    """
    Copies a limited number of images from source to destination for each class.
    Parameters:
        src_dir (str): Source directory where the images are stored.
        dest_dir (str): Destination directory to store the limited images.
        max_images (int): Maximum number of images per class.
    """
    # Ensure destination directory exists
    if not os.path.exists(dest_dir):
        os.makedirs(dest_dir)

    # Iterate through each class folder in the source directory
    for category in os.listdir(src_dir):
        category_path = os.path.join(src_dir, category)
        dest_category_path = os.path.join(dest_dir, category)

        if not os.path.exists(dest_category_path):
            os.makedirs(dest_category_path)

        # Get list of images
        images = [
            os.path.join(category_path, img)
            for img in os.listdir(category_path)
            if img.endswith(('.jpeg', '.jpg', '.png'))
        ]

        # Shuffle and select limited images
        random.shuffle(images)
        limited_images = images[:max_images]

        # Copy limited images to destination folder
        for img_path in limited_images:
            shutil.copy(img_path, os.path.join(dest_category_path, os.path.basename(img_path)))

# Source and destination paths
src_train_dir = '/content/OCT_Dataset/OCT2017 /train'
dest_train_dir = '/content/OCT_Dataset/OCT2017 /train_limited'

# Apply the function
limit_images_per_class(src_train_dir, dest_train_dir, max_images=8000)

print("Training data limited to 8,000 images per class.")


Training data limited to 8,000 images per class.


In [4]:
train_dir = '/content/OCT_Dataset/OCT2017 /train_limited'
val_dir = '/content/OCT_Dataset/OCT2017 /val'
test_dir = '/content/OCT_Dataset/OCT2017 /test'


Define Data Generators

In [5]:
# Define image dimensions and batch size
img_height, img_width = 224, 224
batch_size = 32

# Training data generator with combined augmentation techniques
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,          # Normalize pixel values
    rotation_range=20,          # Random rotation (enhanced from 15)
    width_shift_range=0.2,      # Random horizontal shifts (enhanced from 0.1)
    height_shift_range=0.2,     # Random vertical shifts (enhanced from 0.1)
    shear_range=0.2,            # Random shearing (added)
    zoom_range=0.2,             # Random zoom (enhanced from 0.1)
    horizontal_flip=True,       # Random horizontal flipping
    fill_mode='nearest',         # Filling mode for shifted pixels
    brightness_range=(0.8, 1.2),  # Brightness adjustment
    channel_shift_range=0.2       # Channel shifting
)

# Validation and testing data generators (only normalization, no augmentation)
val_test_datagen = ImageDataGenerator(rescale=1.0 / 255)

# Training data generator
train_generator = train_datagen.flow_from_directory(
    train_dir,                  # Training directory
    target_size=(img_height, img_width),  # Resize images
    batch_size=batch_size,      # Batch size
    class_mode='categorical'    # For multi-class classification
)

# Validation data generator
val_generator = val_test_datagen.flow_from_directory(
    val_dir,                    # Validation directory
    target_size=(img_height, img_width),  # Resize images
    batch_size=batch_size,      # Batch size
    class_mode='categorical'    # For multi-class classification
)

# Test data generator
test_generator = val_test_datagen.flow_from_directory(
    test_dir,               # Testing directory
    target_size=(img_height, img_width),  # Resize images
    batch_size=batch_size,      # Batch size
    class_mode='categorical'    # For multi-class classification
)

print("Enhanced data generators are ready!")


Found 32000 images belonging to 4 classes.
Found 32 images belonging to 4 classes.
Found 968 images belonging to 4 classes.
Enhanced data generators are ready!


4. Train Individual Models

In [6]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import layers, models, optimizers

def fine_tune_efficientnet():
    base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

    # Unfreeze some layers for fine-tuning
    base_model.trainable = True
    for layer in base_model.layers[:200]:  # Freeze first 200 layers (adjustable)
        layer.trainable = False

    x = base_model.output
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(512, activation='relu')(x)
    x = layers.Dropout(0.5)(x)  # Dropout for regularization
    output = layers.Dense(4, activation='softmax')(x)

    model = models.Model(inputs=base_model.input, outputs=output)

    # Compile with a lower learning rate
    optimizer = optimizers.Adam(learning_rate=1e-4)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

efficientnet_model = fine_tune_efficientnet()
history_efficientnet = efficientnet_model.fit(train_generator, validation_data=val_generator, epochs=12)
efficientnet_model.save('fine_tuned_efficientnet.h5')



16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
Epoch 1/12


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1000/1000 ━━━━━━━━━━━━━━━━━━━━ 616s 577ms/step - accuracy: 0.2535 - loss: 1.4111 - val_accuracy: 0.2500 - val_loss: 1.3869
Epoch 2/12
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 578s 552ms/step - accuracy: 0.2499 - loss: 1.3907 - val_accuracy: 0.2500 - val_loss: 1.3868
Epoch 3/12
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 557s 547ms/step - accuracy: 0.2535 - loss: 1.3871 - val_accuracy: 0.2500 - val_loss: 1.3864
Epoch 4/12
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 563s 549ms/step - accuracy: 0.2505 - loss: 1.3872 - val_accuracy: 0.2500 - val_loss: 1.3864
Epoch 5/12
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 552s 540ms/step - accuracy: 0.2574 - loss: 1.3874 - val_accuracy: 0.2500 - val_loss: 1.3865
Epoch 6/12
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 543s 540ms/step - accuracy: 0.2505 - loss: 1.3866 - val_accuracy: 0.2500 - val_loss: 1.3863
Epoch 7/12
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 540s 537ms/step - accuracy: 0.2490 - loss: 1.3864 - val_accuracy: 0.2500 - val_loss: 1.3870
Epoch 8/12
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 538s 535ms/step - accuracy: 0.2

In [7]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, models, optimizers

def fine_tune_resnet():
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

    # Unfreeze last few layers for fine-tuning
    for layer in base_model.layers[:143]:  # Freeze first 143 layers (adjustable)
        layer.trainable = False
    for layer in base_model.layers[143:]:
        layer.trainable = True

    x = base_model.output
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(512, activation='relu')(x)
    x = layers.Dropout(0.5)(x)  # Dropout for regularization
    output = layers.Dense(4, activation='softmax')(x)

    model = models.Model(inputs=base_model.input, outputs=output)

    # Compile with a low learning rate for fine-tuning
    optimizer = optimizers.Adam(learning_rate=1e-4)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

resnet_model = fine_tune_resnet()
history_resnet = resnet_model.fit(train_generator, validation_data=val_generator, epochs=12)
resnet_model.save('fine_tuned_resnet.h5')


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step
Epoch 1/12
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 592s 564ms/step - accuracy: 0.3914 - loss: 1.2745 - val_accuracy: 0.3125 - val_loss: 5.8072
Epoch 2/12
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 578s 575ms/step - accuracy: 0.5320 - loss: 1.0769 - val_accuracy: 0.5938 - val_loss: 0.9632
Epoch 3/12
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 616s 568ms/step - accuracy: 0.5742 - loss: 0.9895 - val_accuracy: 0.4375 - val_loss: 1.4035
Epoch 4/12
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 610s 557ms/step - accuracy: 0.6012 - loss: 0.9422 - val_accuracy: 0.5312 - val_loss: 1.0157
Epoch 5/12
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 558s 554ms/step - accuracy: 0.6258 - loss: 0.8929 - val_accuracy: 0.5938 - val_loss: 1.8408
Epoch 6/12
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 555s 548ms/step - accuracy: 0.6385 - loss: 0.8729 - val_accuracy: 0.8125 - val_loss: 0.6564
Epoch 7/12
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 570s 556ms/step - accuracy: 0.6526 - loss: 0.8380 - val_accuracy: 0.6562 - val_loss: 0.8641
Epoch 8/12


In [ ]:
from tensorflow.keras.applications import DenseNet121

def create_densenet():
    base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    base_model.trainable = False

    x = base_model.output
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(256, activation='relu')(x)
    output = tf.keras.layers.Dense(4, activation='softmax')(x)

    model = tf.keras.models.Model(inputs=base_model.input, outputs=output)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

densenet_model = create_densenet()
history_densenet = densenet_model.fit(train_generator, validation_data=val_generator, epochs=12)
densenet_model.save('densenet_model.h5')


29084464/29084464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Epoch 1/12


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1000/1000 ━━━━━━━━━━━━━━━━━━━━ 661s 627ms/step - accuracy: 0.6589 - loss: 0.8312 - val_accuracy: 0.9062 - val_loss: 0.3138
Epoch 2/12
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 647s 608ms/step - accuracy: 0.7702 - loss: 0.6000 - val_accuracy: 0.9062 - val_loss: 0.3083
Epoch 3/12
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 620s 606ms/step - accuracy: 0.7847 - loss: 0.5611 - val_accuracy: 0.9062 - val_loss: 0.2697
Epoch 4/12
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 613s 599ms/step - accuracy: 0.7859 - loss: 0.5491 - val_accuracy: 0.8438 - val_loss: 0.3913
Epoch 5/12
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 604s 599ms/step - accuracy: 0.7971 - loss: 0.5278 - val_accuracy: 0.9375 - val_loss: 0.2154
Epoch 6/12
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 618s 597ms/step - accuracy: 0.7963 - loss: 0.5368 - val_accuracy: 0.8750 - val_loss: 0.2852
Epoch 7/12
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 614s 608ms/step - accuracy: 0.8041 - loss: 0.5105 - val_accuracy: 0.9062 - val_loss: 0.2050
Epoch 8/12
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 603s 592ms/step - accuracy: 0.8

In [ ]:
# Assuming you have your test_generator defined as before
test_loss, test_accuracy = densenet_model.evaluate(test_generator)

print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

31/31 ━━━━━━━━━━━━━━━━━━━━ 14s 449ms/step - accuracy: 0.9426 - loss: 0.1915
Test Loss: 0.1857
Test Accuracy: 0.9432


5. Create and Train the Ensemble

In [8]:
# Load trained models
efficientnet_model = load_model('fine_tuned_efficientnet.h5')
resnet_model = load_model('fine_tuned_resnet.h5')
densenet_model = load_model('densenet_model.h5')

# Freeze all models
for model in [efficientnet_model, resnet_model, densenet_model]:
    model.trainable = False

# Create ensemble output
efficientnet_output = efficientnet_model.output
resnet_output = resnet_model.output
densenet_output = densenet_model.output

ensemble_output = Average()([efficientnet_output, resnet_output, densenet_output])

# Create ensemble model
ensemble_model = Model(
    inputs=[efficientnet_model.input, resnet_model.input, densenet_model.input],
    outputs=ensemble_output
)

ensemble_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the ensemble (optional fine-tuning)
history_ensemble = ensemble_model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=5
)

ensemble_model.save('ensemble_model.h5')


FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = 'efficientnet_model.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

Evaluate the Ensemble

In [ ]:
test_loss, test_accuracy = ensemble_model.evaluate(test_generator)
print(f"Ensemble Test Accuracy: {test_accuracy:.2f}")